## Searching for bouts for a day of ephys recording
- mircophone chan and sync data and bouts are extracted in sglx_preprocess-s_b1253_21-multisess-pouli-ephys

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2021-11-19 10:37:15,434 root         INFO     Running on pouli


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs

### Get the file locations for a session (day) of recordings

In [3]:
reload(et)
sess_par = {'bird': 'z_w12m7_20',
           'sess': '2020-11-06',
           'sort': 2}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='sglx')

raw_folder = exp_struct['folders']['sglx']

derived_folder = exp_struct['folders']['derived']

bouts_folder = os.path.join(os.path.split(derived_folder)[0], 'bouts_sglx')

sess_bouts_file = os.path.join(bouts_folder, 'bout_sess_auto.pickle')
sess_bouts_curated_file = os.path.join(bouts_folder, 'bout_curated.pickle')

#os.makedirs(bouts_folder, exist_ok=True)

In [4]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/z_w12m7_20',
 'raw': '/mnt/sphere/speech_bci/raw_data/z_w12m7_20/2020-11-06',
 'sglx': '/mnt/sphere/speech_bci/raw_data/z_w12m7_20/2020-11-06/sglx',
 'kwik': '/experiment/z_w12m7_20/sglx/kwik/2020-11-06',
 'processed': '/mnt/sphere/speech_bci/processed_data/z_w12m7_20/2020-11-06/sglx',
 'derived': '/mnt/sphere/speech_bci/derived_data/z_w12m7_20/2020-11-06/sglx',
 'tmp': '/experiment/tmp/tmp',
 'msort': '/experiment/tmp/z_w12m7_20/sglx/msort/2020-11-06',
 'ksort': '/experiment/tmp/z_w12m7_20/sglx/ksort/2020-11-06/',
 'sort': '/mnt/sphere/speech_bci/derived_data/z_w12m7_20/2020-11-06/sglx/'}

### load concatenated the files of the session

In [5]:
def read_session_auto_bouts(exp_struct):
    # list all files of the session
    # read into list of pandas dataframes and concatenate
    # read the search parameters of the first session
    # return the big pd and the search params
    derived_folder = exp_struct['folders']['derived']
    
    search_params_files = et.get_sgl_files_epochs(derived_folder, file_filter='bout_search_params.pickle')
    hparams=None
    with open(search_params_files[0], 'rb') as fh:
        hparams = pickle.load(fh)
    
    sess_bout_pd_file = os.path.join(os.path.split(derived_folder)[0], 'bouts_sglx', 'bout_auto.pickle')
    try:
        bout_pd = pd.read_pickle(sess_bout_pd_file)
    except FileNotFoundError:
        logger.info('Not found session pandas file {}, loading each epoch file and concatenating'.format(sess_bout_pd_file))
        bout_pd_files = et.get_sgl_files_epochs(derived_folder, file_filter='bout_auto.pickle')
        logger.info('Files to load {}'.format(bout_pd_files))
        bout_pd = pd.concat([pd.read_pickle(p) for p in bout_pd_files[:]])
    
    bout_pd.reset_index(inplace=True, drop=True)
    return bout_pd, hparams

bout_pd, hparams = read_session_auto_bouts(exp_struct)

In [6]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_ms      193 non-null    int64  
 1   end_ms        193 non-null    int64  
 2   start_sample  193 non-null    int64  
 3   end_sample    193 non-null    int64  
 4   p_step        193 non-null    object 
 5   rms_p         193 non-null    float64
 6   peak_p        193 non-null    float64
 7   bout_check    193 non-null    bool   
 8   file          193 non-null    object 
 9   len_ms        193 non-null    int64  
 10  syl_in        193 non-null    object 
 11  n_syl         193 non-null    int64  
 12  peaks_p       193 non-null    object 
 13  n_peaks       193 non-null    int64  
 14  l_p_ratio     186 non-null    float64
 15  waveform      193 non-null    object 
 16  confusing     193 non-null    bool   
dtypes: bool(2), float64(3), int64(7), object(5)
memory usage: 23.1+ KB


In [7]:
np.unique(bout_pd['start_ms']).shape

(193,)

In [8]:
### list all the source files
sess_wav_files = np.unique(bout_pd['file'].values)
sess_wav_files

array(['/mnt/sphere/speech_bci/derived_data/z_w12m7_20/2020-11-06/sglx/dir_g0/wav_mic.wav'],
      dtype=object)

#### optional: drop the night recordings

In [9]:
# night_file = sess_wav_files[-1]
# bout_pd.drop(bout_pd[bout_pd['file']==night_file].index, inplace=True)
#print(night_file)

In [10]:
np.unique(bout_pd['file'].values)

array(['/mnt/sphere/speech_bci/derived_data/z_w12m7_20/2020-11-06/sglx/dir_g0/wav_mic.wav'],
      dtype=object)

In [11]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_ms      193 non-null    int64  
 1   end_ms        193 non-null    int64  
 2   start_sample  193 non-null    int64  
 3   end_sample    193 non-null    int64  
 4   p_step        193 non-null    object 
 5   rms_p         193 non-null    float64
 6   peak_p        193 non-null    float64
 7   bout_check    193 non-null    bool   
 8   file          193 non-null    object 
 9   len_ms        193 non-null    int64  
 10  syl_in        193 non-null    object 
 11  n_syl         193 non-null    int64  
 12  peaks_p       193 non-null    object 
 13  n_peaks       193 non-null    int64  
 14  l_p_ratio     186 non-null    float64
 15  waveform      193 non-null    object 
 16  confusing     193 non-null    bool   
dtypes: bool(2), float64(3), int64(7), object(5)
memory usage: 23.1+ KB


In [12]:
np.unique(bout_pd['start_ms']).size

193

###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

In [13]:
def sample_rate_from_wav(wav_path):
    x, sample_rate = wavfile.read(wav_path)
    return sample_rate

if hparams['sample_rate'] is None:
    one_wav_path = bpd.loc[0, 'file']
    logger.info('Sample rate not saved in parameters dict, searching it in ' + one_wav_path)
    hparams['sample_rate'] = sample_rate_from_wav(one_wav_path)

In [14]:
def cleanup(bout_pd: pd.DataFrame):
    ## check for empty waveforms (how woudld THAT happen???)
    bout_pd['valid_waveform'] = bout_pd['waveform'].apply(lambda x: (False if x.size==0 else True))
    
    # valid is & of all the validated criteria
    bout_pd['valid'] = bout_pd['valid_waveform']
    
    ## fill in the epoch
    bout_pd['epoch'] = bout_pd['file'].apply(lambda x: et.split_path(x)[-2])
    
    # drop not valid and reset index
    bout_pd.drop(bout_pd[bout_pd['valid']==False].index, inplace=True)
    bout_pd.reset_index(drop=True, inplace=True)
    
    # set all to 'confusing' (unchecked) and 'bout_check' false (not a bout)
    bout_pd['confusing'] = True
    bout_pd['bout_check'] = False

cleanup(bout_pd)

In [15]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   start_ms        193 non-null    int64  
 1   end_ms          193 non-null    int64  
 2   start_sample    193 non-null    int64  
 3   end_sample      193 non-null    int64  
 4   p_step          193 non-null    object 
 5   rms_p           193 non-null    float64
 6   peak_p          193 non-null    float64
 7   bout_check      193 non-null    bool   
 8   file            193 non-null    object 
 9   len_ms          193 non-null    int64  
 10  syl_in          193 non-null    object 
 11  n_syl           193 non-null    int64  
 12  peaks_p         193 non-null    object 
 13  n_peaks         193 non-null    int64  
 14  l_p_ratio       186 non-null    float64
 15  waveform        193 non-null    object 
 16  confusing       193 non-null    bool   
 17  valid_waveform  193 non-null    boo

In [16]:
np.unique(bout_pd['start_ms']).shape

(193,)

In [17]:
bout_pd

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch
0,3496163,3499168,87404075,87479200,"[365.2130651692613, 21.52861785064727, 27.9565...",70.924488,365.213065,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,3005,"[[3496163, 3496248], [3497403, 3497493], [3497...",7,"[277, 306, 317, 322, 443, 450, 573, 578, 591]",9,333.888889,"[3113, 3114, 3092, 3065, 3053, 2939, 2956, 313...",True,True,True,dir_g0
1,3516398,3517633,87909950,87940825,"[24.774312753338904, 21.341322813100255, 22.80...",70.924488,75.910504,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,1235,"[[3516398, 3516823], [3516853, 3517633]]",2,[232],1,1235.000000,"[1633, 1576, 1584, 1547, 1632, 1666, 1575, 155...",True,True,True,dir_g0
2,3523963,3531278,88099075,88281950,"[28.109543390680244, 31.73958122888766, 31.890...",70.924488,40.591913,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,7315,"[[3523963, 3524073], [3526638, 3526738], [3528...",5,"[4, 544, 900]",3,2438.333333,"[8037, 7938, 7969, 7962, 7985, 8029, 7946, 794...",True,True,True,dir_g0
3,3538288,3542738,88457200,88568450,"[31.551787644736283, 21.230169338530395, 21.19...",70.924488,46.247367,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,4450,"[[3538288, 3538393], [3541288, 3541868], [3541...",4,"[19, 709, 843]",3,1483.333333,"[-4553, -4606, -4624, -4603, -4616, -4664, -46...",True,True,True,dir_g0
4,3548518,3564458,88712950,89111450,"[22.69540114152416, 29.99044562858022, 28.0426...",70.924488,42.420410,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,15940,"[[3548518, 3548568], [3548793, 3548953], [3550...",9,"[858, 868, 1330, 1774, 2308, 2708]",6,2656.666667,"[2208, 2201, 2050, 2153, 2128, 2199, 2107, 208...",True,True,True,dir_g0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,6948628,6948758,173715700,173718950,"[20.80006076339254, 19.650147250971585, 20.174...",70.924488,39.440554,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,130,"[[6948628, 6948758]]",1,[5],1,130.000000,"[2357, 2298, 2305, 2303, 2309, 2355, 2272, 232...",True,True,True,dir_g0
189,6961043,6961138,174026075,174028450,"[24.3514303003263, 18.507237825361223, 23.3256...",70.924488,39.151204,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,95,"[[6961043, 6961138]]",1,[7],1,95.000000,"[-1003, -973, -865, -916, -900, -844, -950, -1...",True,True,True,dir_g0
190,6978553,6980813,174463825,174520325,"[25.1208058938151, 21.24865622293375, 20.29980...",70.924488,42.780563,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,2260,"[[6978553, 6978783], [6980588, 6980813]]",2,"[27, 430]",2,1130.000000,"[1169, 1236, 1195, 1237, 1170, 1043, 1094, 109...",True,True,True,dir_g0
191,6987738,6987803,174693450,174695075,"[26.671553523831427, 29.949003832174395, 24.35...",70.924488,29.949004,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,65,"[[6987738, 6987803]]",1,[],0,NaN,"[-4052, -4042, -4073, -4086, -4125, -4032, -40...",True,True,True,dir_g0


In [18]:
reload(et)

<module 'ceciestunepipe.file.bcistructure' from '/home/finch/repos/ceciestunepipe/ceciestunepipe/file/bcistructure.py'>

#### compute the spectrograms

In [19]:
# but don't compute if they were already computed
# try:
#     bout_pd = pd.read_pickle(sess_bouts_file) ## make it jump if there is no sess_bouts file
#     one_spec = bout_pd['spectrogram'][0] ## make it jump if there is no spectrogram key
#     logger.info('loaded bout pandas with spectrograms from ' + sess_bouts_file)
# except KeyError:
#     logger.info('Computing spectrograms')
#     bout_pd['spectrogram'] = bout_pd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
#     logger.info('saving bout pandas with spectrogram to ' + sess_bouts_file)
#     bout_pd.to_pickle(sess_bouts_file)

In [20]:
# just compute them alright?
logger.info('Computing spectrograms')
bout_pd['spectrogram'] = bout_pd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
logger.info('saving bout pandas with spectrogram to ' + sess_bouts_file)
bout_pd.to_pickle(sess_bouts_file)

2021-11-19 10:37:21,087 root         INFO     Computing spectrograms
2021-11-19 10:37:37,010 root         INFO     saving bout pandas with spectrogram to /mnt/sphere/speech_bci/derived_data/z_w12m7_20/2020-11-06/bouts_sglx/bout_sess_auto.pickle


In [21]:
bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram
0,3496163,3499168,87404075,87479200,"[365.2130651692613, 21.52861785064727, 27.9565...",70.924488,365.213065,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,3005,...,7,"[277, 306, 317, 322, 443, 450, 573, 578, 591]",9,333.888889,"[3113, 3114, 3092, 3065, 3053, 2939, 2956, 313...",True,True,True,dir_g0,"[[0.26846311722804306, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,3516398,3517633,87909950,87940825,"[24.774312753338904, 21.341322813100255, 22.80...",70.924488,75.910504,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,1235,...,2,[232],1,1235.000000,"[1633, 1576, 1584, 1547, 1632, 1666, 1575, 155...",True,True,True,dir_g0,"[[0.16286924726281934, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [22]:
bout_pd['file'][0]

'/mnt/sphere/speech_bci/derived_data/z_w12m7_20/2020-11-06/sglx/dir_g0/wav_mic.wav'

## inspect the bouts and curate them

#### visualize one bout

In [23]:
bout_pd.iloc[0]

start_ms                                                    3496163
end_ms                                                      3499168
start_sample                                               87404075
end_sample                                                 87479200
p_step            [365.2130651692613, 21.52861785064727, 27.9565...
rms_p                                                     70.924488
peak_p                                                   365.213065
bout_check                                                    False
file              /mnt/sphere/speech_bci/derived_data/z_w12m7_20...
len_ms                                                         3005
syl_in            [[3496163, 3496248], [3497403, 3497493], [3497...
n_syl                                                             7
peaks_p               [277, 306, 317, 322, 443, 450, 573, 578, 591]
n_peaks                                                           9
l_p_ratio                                       

In [24]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [25]:
def viz_one_bout(df: pd.Series, sub_sample=1):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    
    
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

viz_one_bout(bout_pd.iloc[24])

    'data': [{'type': 'scatter',
              'uid': 'e7ddce93-073c-4577-b553-c…

In [26]:
bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram
0,3496163,3499168,87404075,87479200,"[365.2130651692613, 21.52861785064727, 27.9565...",70.924488,365.213065,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,3005,...,7,"[277, 306, 317, 322, 443, 450, 573, 578, 591]",9,333.888889,"[3113, 3114, 3092, 3065, 3053, 2939, 2956, 313...",True,True,True,dir_g0,"[[0.26846311722804306, 0.0, 0.0, 0.0, 0.0, 0.0..."
1,3516398,3517633,87909950,87940825,"[24.774312753338904, 21.341322813100255, 22.80...",70.924488,75.910504,False,/mnt/sphere/speech_bci/derived_data/z_w12m7_20...,1235,...,2,[232],1,1235.000000,"[1633, 1576, 1584, 1547, 1632, 1666, 1575, 155...",True,True,True,dir_g0,"[[0.16286924726281934, 0.0, 0.0, 0.0, 0.0, 0.0..."


## use it in a widget


#### add a 'confusing' label, for not/sure/mixed.
we want to avoid having things we are not sure of in the training dataset

In [27]:
bout_pd.reset_index(drop=True, inplace=True)

In [28]:
## Set confusing by default, will only be False once asserted bout/or not
bout_pd['confusing'] = True
bout_pd['bout_check'] = False

In [29]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [30]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_confusing = None
        
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_confusing = widgets.Checkbox(description='Not sure or mixed')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Check", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Uncheck", button_style='danger',
                                                 icon='wrong')
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        left_box = widgets.VBox([self.buttons['Prev'], self.buttons['Uncheck']])
        right_box = widgets.VBox([self.buttons['Next'], self.buttons['Check']])
        button_box = widgets.HBox([left_box, right_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1,step=1, 
                                        description="Bout candidate index")
        
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing]),
                                    self.m_pick])
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            curr_bout.value -= 1
        elif button.description == 'Check':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        elif button.description == 'Uncheck':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
        
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bout_pd)

### save it

In [31]:
raise KeyboardInterrupt

KeyboardInterrupt: 

In [ ]:
hparams

In [ ]:
### get the curated file path
##save to the curated file path
viz_bout.bouts_pd.to_pickle(sess_bouts_curated_file)
logger.info('saved curated bout pandas to pickle {}'.format(sess_bouts_curated_file))

### summary of where the good bouts are along the session

In [ ]:
def give_summary(bpd: pd.DataFrame):
    all_files = np.unique(bpd['file'])
    summary_dict = {f: np.where((bpd['bout_check']==True) & (bpd['file']==f))[0].size for f in all_files}
    return summary_dict

sum_dict = give_summary(viz_bout.bouts_pd)
sum_dict

## Some helper functions for somewhere down the line

In [ ]:
sess_par_e = dict.copy(sess_par)
sess_par_e['epoch'] = '0610_g0' 

In [ ]:
bpd.head()

In [ ]:
reload(et)
bpd = et.get_epoch_bout_pd(sess_par_e, only_curated=True)
bpd.head()

In [ ]:
bpd.head(1)

In [ ]:
bpd['start_ms'].values

In [ ]:
bpd['end_ms'].values - bpd['start_ms'].values

In [ ]:
bpd